# RESIDUAL NEURAL NETWORK: RESNET-50

Here we try to build the model architecutre of ResNet-50

## IMPORTS

***mount drive***

In [1]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


***general imports***

In [2]:
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib.pyplot import imshow 
import pydot 
import scipy.misc
from IPython.display import SVG

***tensorflow and keras imports***

In [3]:
import tensorflow as tf
from keras import layers 

from keras.layers import  Input, Add, Dense, Activation, ZeroPadding2D,\
                          BatchNormalization, Flatten, Conv2D, AveragePooling2D,\
                          MaxPooling2D, GlobalMaxPooling2D

from keras.models import Model, load_model
from keras.preprocessing import image 
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input 
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.initializers import glorot_uniform

## LOAD DATA

we have four types of data. they consist of:
- nut 
- washer
- bolt
- pin

***loop over images and load them***

In [4]:
import cv2

# setting the path 
dataset_path = "/content/drive/MyDrive/Youtube/2 - ResNet50/data/"

# finding the classes in dataset
classes = os.listdir(dataset_path)

im_size = (64, 64)
images=[]
labels=[]

# loop over all of the images --> loading each image --> resizing each image
for class_ in classes:
  class_path = dataset_path + str(class_)
  filenames = [i for i in os.listdir(class_path)]

  for filename in filenames:
    image = cv2.imread(class_path + "/" + filename)
    image = cv2.resize(image, im_size)

    images.append(image)
    labels.append(str(class_))

labels = np.array(labels)
labels.shape

(226,)

## PRE-PROCESSING DATA

***image to numpy array***

In [5]:
images = np.array(images)
images.shape

(226, 64, 64, 3)

***normalize***

In [6]:
images = images.astype("float32")/255.0

***one-hot encoding***

In [15]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# converting to numpy array
labels = np.array(labels)

# converting labels from string to numbers 
labelencoder = LabelEncoder()
labels = labelencoder.fit_transform(labels)

# add another dimension
labels = labels.reshape(-1,1)

# create an instance of one-hot encoder
onehotencoder=OneHotEncoder(categories='auto', sparse=False)

# apply the one-hot encoder to the labels
labels = onehotencoder.fit_transform(labels)

print(labels.shape)

ValueError: ignored

***shuffle and test/train split***

In [16]:
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split

# shuffle data
images, labels = shuffle(images, labels, random_state=69)

# train/test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=420)

print(X_train.shape)
print(y_train.shape)

(180, 64, 64, 3)
(180, 4)


## MODEL ARCHITECTURE

***build identity block***

In [17]:
def identity_block(X, kernel_size, filters, stage, block):

  conv_name_base = "res" + str(stage) + block + "_branch"
  bn_name_base = "bn" + str(stage) + block + "_branch"

  F1, F2, F3 = filters

  X_shortcut = X

  X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F2, kernel_size = (kernel_size, kernel_size), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

***build convolutional block***

In [18]:
def convolutional_block(X, kernel_size, filters, stage, block, s = 2):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X) 
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)  
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size = (kernel_size, kernel_size), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

***build model structure***

In [19]:
def ResNet50(input_shape=(64, 64, 3), classes=4):
    
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input) 

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X)  
    X = BatchNormalization(axis=3, name='bn_conv1')(X) 
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) 

    X = convolutional_block(X, kernel_size=3, filters=[64, 64, 256], stage=2, block='a', s=1)
   
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b') 
    
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # ----------
    
    X = convolutional_block(X, kernel_size = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, kernel_size = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
     
    X = convolutional_block(X, kernel_size = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D((2,2), name="avg_pool")(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

## COMPILE AND TRAIN MODEL

***create an instance of the built model***

In [20]:
model = ResNet50(input_shape = (64, 64, 3), classes=4)

model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 70, 70, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 32, 32, 64)   9472        ['zero_padding2d_1[0][0]']       
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1[0][0]']           

***compile the model***

In [21]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

***train the model***

In [22]:
history = model.fit(X_train, y_train, epochs = 20, batch_size=16)

Epoch 1/20
12/12 [==============================] - 43s 118ms/step - loss: 2.2946 - accuracy: 0.6222
Epoch 2/20
12/12 [==============================] - 1s 56ms/step - loss: 1.8489 - accuracy: 0.7333
Epoch 3/20
12/12 [==============================] - 1s 59ms/step - loss: 1.6040 - accuracy: 0.8222
Epoch 4/20
12/12 [==============================] - 1s 59ms/step - loss: 0.6015 - accuracy: 0.8500
Epoch 5/20
12/12 [==============================] - 1s 62ms/step - loss: 0.4596 - accuracy: 0.9167
Epoch 6/20
12/12 [==============================] - 1s 59ms/step - loss: 0.2657 - accuracy: 0.9000
Epoch 7/20
12/12 [==============================] - 1s 43ms/step - loss: 0.3187 - accuracy: 0.9500
Epoch 8/20
12/12 [==============================] - 1s 43ms/step - loss: 0.2391 - accuracy: 0.9333
Epoch 9/20
12/12 [==============================] - 1s 42ms/step - loss: 0.4379 - accuracy: 0.9500
Epoch 10/20
12/12 [==============================] - 1s 43ms/step - loss: 0.3426 - accuracy: 0.9056
Epoch 1

## EVALUATE MODEL PERFORMANCE

***use test data for evaluation***

In [23]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 2s 343ms/step - loss: 1.7283 - accuracy: 0.0435


[1.7283012866973877, 0.043478261679410934]